# Depth estimation of a Pytorch model 

The below procedure helps converting a Pytorch model which is taken from original paper [here](https://syncedreview.com/2020/04/13/ai-transforms-rgb-d-images-into-an-impressive-3d-format/) to TensorFlow Lite so the model can be used inside an android application. The Pytorch model's code is taken from [this](https://github.com/vt-vl-lab/3d-photo-inpainting) github repository. This model takes as input an RGB image of shape [1, 3, 384, 384] and outputs a float array of shape [1, 1, 384, 384] which is converted to a grayscale image. Inference with the TensorFlow Lite model takes place with the same procedure.

### First load necessary imports

In [1]:
!pip install onnx
!pip install onnxruntime
!pip install pip install git+https://github.com/onnx/onnx-tensorflow.git


import gdown
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torchvision import models
from collections import OrderedDict
import onnx
import onnxruntime
from onnx_tf.backend import prepare

import re
import numpy as np
import cv2
import torch
import os

import tensorflow as tf

     |████████████████████████████████| 14.5MB 268kB/s 
     |████████████████████████████████| 4.1MB 2.9MB/s 
  Cloning https://github.com/onnx/onnx-tensorflow.git to /tmp/pip-req-build-77100chb
  Running command git clone -q https://github.com/onnx/onnx-tensorflow.git /tmp/pip-req-build-77100chb
     |████████████████████████████████| 706kB 3.0MB/s 
  Created wheel for onnx-tf: filename=onnx_tf-1.7.0-cp37-none-any.whl size=219452 sha256=79f487205a95c4a2ba7f91d2b0dcc2dc3873350a37c9bff3ee7790d0f99eb4d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-sc9g9djc/wheels/54/24/31/8873b33d2d560efdfa1ed6f346df67ef793b1897358705a480
Successfully built onnx-tf


### Helper function to load the model's weights

In [2]:
def copyStateDict(state_dict):
    if list(state_dict.keys())[0].startswith("module"):
        start_idx = 1
    else:
        start_idx = 0
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = ".".join(k.split(".")[start_idx:])
        new_state_dict[name] = v
    return new_state_dict

### Download model's weights

In [3]:
!wget https://filebox.ece.vt.edu/~jbhuang/project/3DPhoto/model/model.pt

--2021-04-01 16:15:14--  https://filebox.ece.vt.edu/~jbhuang/project/3DPhoto/model/model.pt
Resolving filebox.ece.vt.edu (filebox.ece.vt.edu)... 128.173.88.43
Connecting to filebox.ece.vt.edu (filebox.ece.vt.edu)|128.173.88.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149751722 (143M)
Saving to: ‘model.pt’

model.pt            100%[===================>] 142.81M  14.5MB/s    in 19s     

2021-04-01 16:15:34 (7.44 MB/s) - ‘model.pt’ saved [149751722/149751722]



### Model's architecture

In [4]:
class MonoDepthNet(nn.Module):
    """Network for monocular depth estimation.
    """

    def __init__(self, path=None, features=256):
        """Init.
        Args:
            path (str, optional): Path to saved model. Defaults to None.
            features (int, optional): Number of features. Defaults to 256.
        """
        super().__init__()

        resnet = models.resnet50(pretrained=False)

        self.pretrained = nn.Module()
        self.scratch = nn.Module()
        self.pretrained.layer1 = nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu,
                                               resnet.maxpool, resnet.layer1)

        self.pretrained.layer2 = resnet.layer2
        self.pretrained.layer3 = resnet.layer3
        self.pretrained.layer4 = resnet.layer4

        # adjust channel number of feature maps
        self.scratch.layer1_rn = nn.Conv2d(256, features, kernel_size=3, stride=1, padding=1, bias=False)
        self.scratch.layer2_rn = nn.Conv2d(512, features, kernel_size=3, stride=1, padding=1, bias=False)
        self.scratch.layer3_rn = nn.Conv2d(1024, features, kernel_size=3, stride=1, padding=1, bias=False)
        self.scratch.layer4_rn = nn.Conv2d(2048, features, kernel_size=3, stride=1, padding=1, bias=False)

        self.scratch.refinenet4 = FeatureFusionBlock(features)
        self.scratch.refinenet3 = FeatureFusionBlock(features)
        self.scratch.refinenet2 = FeatureFusionBlock(features)
        self.scratch.refinenet1 = FeatureFusionBlock(features)

        # adaptive output module: 2 convolutions and upsampling
        self.scratch.output_conv = nn.Sequential(nn.Conv2d(features, 128, kernel_size=3, stride=1, padding=1),
                                                 nn.Conv2d(128, 1, kernel_size=3, stride=1, padding=1),
                                                 Interpolate(scale_factor=2, mode='bilinear'))

        # load model
        if path:
            self.load(path)

    def forward(self, x):
        """Forward pass.
        Args:
            x (tensor): input data (image)
        Returns:
            tensor: depth
        """
        layer_1 = self.pretrained.layer1(x)
        layer_2 = self.pretrained.layer2(layer_1)
        layer_3 = self.pretrained.layer3(layer_2)
        layer_4 = self.pretrained.layer4(layer_3)

        layer_1_rn = self.scratch.layer1_rn(layer_1)
        layer_2_rn = self.scratch.layer2_rn(layer_2)
        layer_3_rn = self.scratch.layer3_rn(layer_3)
        layer_4_rn = self.scratch.layer4_rn(layer_4)

        path_4 = self.scratch.refinenet4(layer_4_rn)
        path_3 = self.scratch.refinenet3(path_4, layer_3_rn)
        path_2 = self.scratch.refinenet2(path_3, layer_2_rn)
        path_1 = self.scratch.refinenet1(path_2, layer_1_rn)

        out = self.scratch.output_conv(path_1)

        return out

    def load(self, path):
        """Load model from file.
        Args:
            path (str): file path
        """
        parameters = torch.load(path)

        self.load_state_dict(parameters)


class Interpolate(nn.Module):
    """Interpolation module.
    """

    def __init__(self, scale_factor, mode):
        """Init.
        Args:
            scale_factor (float): scaling
            mode (str): interpolation mode
        """
        super(Interpolate, self).__init__()

        self.interp = nn.functional.interpolate
        self.scale_factor = scale_factor
        self.mode = mode

    def forward(self, x):
        """Forward pass.
        Args:
            x (tensor): input
        Returns:
            tensor: interpolated data
        """
        x = self.interp(x, scale_factor=self.scale_factor, mode=self.mode, align_corners=False)

        return x


class ResidualConvUnit(nn.Module):
    """Residual convolution module.
    """

    def __init__(self, features):
        """Init.
        Args:
            features (int): number of features
        """
        super().__init__()

        self.conv1 = nn.Conv2d(features, features, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv2 = nn.Conv2d(features, features, kernel_size=3, stride=1, padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        """Forward pass.
        Args:
            x (tensor): input
        Returns:
            tensor: output
        """
        out = self.relu(x)
        out = self.conv1(out)
        out = self.relu(out)
        out = self.conv2(out)

        return out + x


class FeatureFusionBlock(nn.Module):
    """Feature fusion block.
    """

    def __init__(self, features):
        """Init.
        Args:
            features (int): number of features
        """
        super().__init__()

        self.resConfUnit = ResidualConvUnit(features)

    def forward(self, *xs):
        """Forward pass.
        Returns:
            tensor: output
        """
        output = xs[0]

        if len(xs) == 2:
            output += self.resConfUnit(xs[1])

        output = self.resConfUnit(output)
        output = nn.functional.interpolate(output, scale_factor=2,
                                           mode='bilinear')

        return output

### Load model's weights and print the architecture

In [5]:

net = MonoDepthNet()
net.load_state_dict(copyStateDict(torch.load('/content/model.pt', map_location='cpu')))

print("Model loaded")
net.eval()

Model loaded


MonoDepthNet(
  (pretrained): Module(
    (layer1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
    

### Pre and post processing of the images

In [10]:

def read_image(path):
    """Read image and output RGB image (0-1).
    Args:
        path (str): path to file
    Returns:
        array: RGB image (0-1)
    """
    img = cv2.imread(path)

    if img.ndim == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 255.0

    return img


def resize_image(img):
    """Resize image and make it fit for network.
    Args:
        img (array): image
    Returns:
        tensor: data ready for network
    """
    height_orig = img.shape[0]
    width_orig = img.shape[1]
    unit_scale = 384.

    if width_orig > height_orig:
        scale = width_orig / unit_scale
    else:
        scale = height_orig / unit_scale

    height = (np.ceil(height_orig / scale / 32) * 32).astype(int)
    width = (np.ceil(width_orig / scale / 32) * 32).astype(int)

    img_resized = cv2.resize(img, (width, height), interpolation=cv2.INTER_LINEAR)

    img_resized = (
        torch.from_numpy(np.transpose(img_resized, (2, 0, 1))).contiguous().float()
    )
    img_resized = img_resized.unsqueeze(0)

    return img_resized


def resize_depth(depth, width, height):
    """Resize depth map and bring to CPU (numpy).
    Args:
        depth (tensor): depth
        width (int): image width
        height (int): image height
    Returns:
        array: processed depth
    """
    print(depth)
    depth = torch.squeeze(depth[0, :, :, :]).to("cpu")
    print(depth.shape)
    depth = cv2.blur(depth.numpy(), (3, 3))
    print(depth)
    depth_resized = cv2.resize(
        depth, (width, height), interpolation=cv2.INTER_LINEAR
    )

    return depth_resized

def write_depth(path, depth, bits=1):
    """Write depth map to pfm and png file.
    Args:
        path (str): filepath without extension
        depth (array): depth
    """
    # write_pfm(path + ".pfm", depth.astype(np.float32))

    depth_min = depth.min()
    depth_max = depth.max()
    print(depth_min)
    print(depth_max)

    max_val = (2**(8*bits))-1
    print(max_val)

    if depth_max - depth_min > np.finfo("float").eps:
        out = max_val * (depth - depth_min) / (depth_max - depth_min)

        # or use 255 * depth as below line
        #out = max_val * depth
    else:
        out = 0

    print(out)

    if bits == 1:
        cv2.imwrite(path + ".png", out.astype("uint8"))
    elif bits == 2:
        cv2.imwrite(path + ".png", out.astype("uint16"))
        
    return

### Load moon.png to use it for inference (this image can be found at images folder of the github's master branch)

In [8]:
batch_size = 1
# Input to the model
# x = torch.randn(batch_size, 3, 384, 384, requires_grad=True)
# onnx_runtime_input = x.detach().numpy()

x = read_image("/content/moon.jpg")
x = resize_image(x)
onnx_runtime_input = x

t1 = datetime.now()
print(x.shape)
print(x)
print(x[0][0][0])
torch_out = net(x)
print("Torch Out shape: {}".format(torch_out.detach().numpy().shape))
t2 = datetime.now()
print("Time taken for Pytoch model", str(t2-t1))
store_out = torch_out[0].detach().numpy()
print(torch_out.detach().numpy())
print("Model ran succesfully")

torch.Size([1, 3, 384, 384])
tensor([[[[0.0108, 0.0059, 0.0210,  ..., 0.0047, 0.0051, 0.0196],
          [0.0266, 0.0265, 0.0184,  ..., 0.0242, 0.0337, 0.0305],
          [0.0025, 0.0064, 0.0145,  ..., 0.0093, 0.0216, 0.0136],
          ...,
          [0.0222, 0.0193, 0.0050,  ..., 0.0011, 0.0360, 0.0257],
          [0.0425, 0.0219, 0.0356,  ..., 0.0489, 0.0025, 0.0018],
          [0.0142, 0.0073, 0.0413,  ..., 0.0246, 0.0053, 0.0399]],

         [[0.0108, 0.0059, 0.0210,  ..., 0.0047, 0.0051, 0.0157],
          [0.0266, 0.0265, 0.0184,  ..., 0.0242, 0.0337, 0.0296],
          [0.0025, 0.0064, 0.0145,  ..., 0.0093, 0.0216, 0.0166],
          ...,
          [0.0222, 0.0193, 0.0050,  ..., 0.0011, 0.0360, 0.0266],
          [0.0425, 0.0219, 0.0356,  ..., 0.0489, 0.0025, 0.0090],
          [0.0142, 0.0073, 0.0419,  ..., 0.0324, 0.0132, 0.0030]],

         [[0.0108, 0.0059, 0.0210,  ..., 0.0047, 0.0051, 0.0452],
          [0.0266, 0.0265, 0.0184,  ..., 0.0242, 0.0337, 0.0401],
          [0.

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Torch Out shape: (1, 1, 384, 384)
Time taken for Pytoch model 0:00:02.526465
[[[[-0.39669645 -0.3981164  -0.4009563  ... -0.38015398 -0.36844572
    -0.3625916 ]
   [-0.40167084 -0.4031891  -0.4062256  ... -0.38171238 -0.37091136
    -0.36551085]
   [-0.41161963 -0.41333452 -0.41676426 ... -0.3848292  -0.3758427
    -0.37134945]
   ...
   [ 1.1138916   1.1200587   1.1323925  ...  1.0947251   1.0689293
     1.0560315 ]
   [ 1.0912312   1.0984172   1.112789   ...  1.086489    1.0595579
     1.0460924 ]
   [ 1.079901    1.0875964   1.1029874  ...  1.0823709   1.0548722
     1.0411228 ]]]]
Model ran succesfully


### Post process the output and write the grayscale image and the numpy array

In [11]:
depth = resize_depth(torch_out.detach(), 480, 640)

filename = os.path.join(
    "/content/", os.path.splitext(os.path.basename("chris_grayscale"))[0]
)
np.save(filename + '.npy', depth)
write_depth(filename, depth, bits=1)

tensor([[[[-0.3967, -0.3981, -0.4010,  ..., -0.3802, -0.3684, -0.3626],
          [-0.4017, -0.4032, -0.4062,  ..., -0.3817, -0.3709, -0.3655],
          [-0.4116, -0.4133, -0.4168,  ..., -0.3848, -0.3758, -0.3713],
          ...,
          [ 1.1139,  1.1201,  1.1324,  ...,  1.0947,  1.0689,  1.0560],
          [ 1.0912,  1.0984,  1.1128,  ...,  1.0865,  1.0596,  1.0461],
          [ 1.0799,  1.0876,  1.1030,  ...,  1.0824,  1.0549,  1.0411]]]])
torch.Size([384, 384])
[[-0.40100303 -0.40199336 -0.40409526 ... -0.37878636 -0.37194005
  -0.3682389 ]
 [-0.404363   -0.405397   -0.40746284 ... -0.37992346 -0.37348303
  -0.3699835 ]
 [-0.41017196 -0.41135567 -0.41353124 ... -0.38226923 -0.37648934
  -0.3733405 ]
 ...
 [ 1.1152345   1.1194822   1.1287322  ...  1.092637    1.0736377
   1.0646838 ]
 [ 1.0996853   1.1043627   1.1144683  ...  1.0839161   1.0655768
   1.0566628 ]
 [ 1.0923584   1.0972621   1.1078731  ...  1.0807909   1.0625161
   1.053476  ]]
-0.44207346
1.4900764
255
[[  5.420366

### Export the model to Onnx format

In [12]:
# Export the model
torch.onnx.export(net,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "3D_depth.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'] # the model's output names
                  )
print("Model converted succesfully")

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/usr/local/lib/python3.7/dist-packages/torch/onnx/symbolic_helper.py:347: UserWarning: You are trying to export the model with onnx:Resize for ONNX opset version 10. This operator might cause results to not match the expected results by PyTorch.
ONNX's Upsample/Resize operator did not match Pytorch's Interpolation until opset 11. Attributes to determine how to transform the input were added in onnx:Resize in opset 11 to support Pytorch's behavior (like coordinate_transformation_mode and nearest_mode).
We recommend using opset 11 and above for models using this operator. 
  "" + str(_export_onnx_opset_version) + ". "


Model converted succesfully


In [13]:
onnx_model = onnx.load("3D_depth.onnx")
onnx.checker.check_model(onnx_model)
print("Model checked succesfully")

Model checked succesfully


In [ ]:
ort_session = onnxruntime.InferenceSession('3D_depth.onnx')

def to_numpy(tensor):
    print(tensor)
    return tensor.detach().cpu().numpy()

ort_inputs = {ort_session.get_inputs()[0].name:onnx_runtime_input}
ort_outs = ort_session.run(None, ort_inputs)
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

### Convert code to TensrFlow 2.0

In [14]:
onnx_model = onnx.load('3D_depth.onnx')
tf_rep = prepare(onnx_model)
tf_rep.export_graph('3D_depth')

print("Model converted to tensorflow graph succesfully.")

INFO:tensorflow:Assets written to: 3D_depth/assets


INFO:tensorflow:Assets written to: 3D_depth/assets


Model converted to tensorflow graph succesfully.


### Convert TensorFlow model to TensorFlow Lite

In [15]:

loaded = tf.saved_model.load('3D_depth')

concrete_func = loaded.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

concrete_func.inputs[0].set_shape([None, 3, 384, 384])

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])

converter.experimental_new_converter = False #error otherwise

converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Uncomment this line for float16 quantization.
#converter.target_spec.supported_types = [tf.float16]

# Uncomment For Integer Quantization
# def representative_data_gen():
#     for file in os.listdir(dataset_path)[:10]:
#         file_path = dataset_path+file
#         image = imgproc.loadImage(file_path)
#         image = cv2.resize(image, dsize=(800, 1280), interpolation=cv2.INTER_LINEAR)
#         img_resized, target_ratio, size_heatmap = imgproc.resize_aspect_ratio(image, 1280, interpolation=cv2.INTER_LINEAR, mag_ratio=1.5)
#         ratio_h = ratio_w = 1 / target_ratio

#         # preprocessing
#         x = imgproc.normalizeMeanVariance(img_resized)
#         x = torch.from_numpy(x).permute(2, 0, 1)    # [h, w, c] to [c, h, w]
#         x = Variable(x.unsqueeze(0))                # [c, h, w] to [b, c, h, w]
#         x = x.cpu().detach().numpy()
#         yield [x]

# converter.representative_dataset = representative_data_gen

tf_lite_model = converter.convert()

open('3D_depth.tflite', 'wb').write(tf_lite_model)

print("Converted to tensorflow lite succesfully.")

Converted to tensorflow lite succesfully.


## Inference with TF Lite

In [16]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="3D_depth.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
#input_shape = input_details[0]['shape']
#input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
image_input = read_image("/content/moon.jpg")
image_input = resize_image(image_input)
print(image_input.shape)
print(image_input[0][0][0])

interpreter.set_tensor(input_details[0]['index'], image_input)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

torch.Size([1, 3, 384, 384])
tensor([0.0108, 0.0059, 0.0210, 0.0098, 0.0013, 0.0213, 0.0248, 0.0127, 0.0100,
        0.0327, 0.0152, 0.0102, 0.0008, 0.0042, 0.0100, 0.0044, 0.0000, 0.0085,
        0.0010, 0.0040, 0.0130, 0.0046, 0.0000, 0.0145, 0.0134, 0.0338, 0.0051,
        0.0121, 0.0257, 0.0153, 0.0073, 0.0175, 0.0076, 0.0157, 0.0121, 0.0053,
        0.0118, 0.0086, 0.0031, 0.0048, 0.0137, 0.0176, 0.0192, 0.0404, 0.0198,
        0.0045, 0.0219, 0.0433, 0.0157, 0.0069, 0.0319, 0.0234, 0.0205, 0.0158,
        0.0095, 0.0474, 0.0195, 0.0251, 0.0068, 0.0024, 0.0100, 0.0175, 0.0226,
        0.0219, 0.0110, 0.0122, 0.0175, 0.0386, 0.0238, 0.0065, 0.0045, 0.0100,
        0.0183, 0.0183, 0.0182, 0.0170, 0.0189, 0.0254, 0.0156, 0.0182, 0.0159,
        0.0284, 0.0265, 0.0111, 0.0114, 0.0070, 0.0165, 0.0229, 0.0352, 0.0133,
        0.0140, 0.0256, 0.0336, 0.0100, 0.0335, 0.0309, 0.0054, 0.0301, 0.0201,
        0.0525, 0.0085, 0.0224, 0.0221, 0.0192, 0.0205, 0.0285, 0.0030, 0.0137,
        0.0

### Post process the TensorFlow Lite ouput and save the final image

In [17]:
depth = resize_depth(torch.from_numpy(output_data), 480, 640)

filename = os.path.join(
    "/content/", os.path.splitext(os.path.basename("chris_tflite"))[0]
)
np.save(filename + '.npy', depth)
write_depth(filename, depth, bits=2)

tensor([[[[-0.2703, -0.2633, -0.2564,  ..., -0.4474, -0.4338, -0.4338],
          [-0.2629, -0.2648, -0.2667,  ..., -0.4520, -0.4314, -0.4314],
          [-0.2555, -0.2663, -0.2770,  ..., -0.4566, -0.4290, -0.4290],
          ...,
          [ 1.1778,  1.1747,  1.1716,  ...,  1.0681,  1.0258,  1.0258],
          [ 1.1385,  1.1350,  1.1315,  ...,  1.0498,  0.9997,  0.9997],
          [ 1.1385,  1.1350,  1.1315,  ...,  1.0498,  0.9997,  0.9997]]]])
torch.Size([384, 384])
[[-0.26465985 -0.26431742 -0.26360548 ... -0.45044664 -0.43826064
  -0.43216765]
 [-0.2641645  -0.26480362 -0.2650305  ... -0.4519883  -0.43824375
  -0.43137145]
 [-0.2633129  -0.26575688 -0.26762018 ... -0.4539023  -0.43785343
  -0.42982897]
 ...
 [ 1.175717    1.1746781   1.1755806  ...  1.0681232   1.039941
   1.0258498 ]
 [ 1.1493714   1.1482494   1.1484095  ...  1.0558846   1.0242572
   1.0084435 ]
 [ 1.1361985   1.135035    1.1348239  ...  1.0497652   1.0164152
   0.9997403 ]]
-0.48215917
1.657558
65535
[[ 6661.543 